#**스마트폰 센서 데이터 기반 모션 분류**
# 단계2 : 기본 모델링


## 0.미션

* 데이터 전처리
    * 가변수화, 데이터 분할, NaN 확인 및 조치, 스케일링 등 필요한 전처리 수행
* 다양한 알고리즘으로 분류 모델 생성
    * 최소 4개 이상의 알고리즘을 적용하여 모델링 수행 
    * 성능 비교
    * 각 모델의 성능을 저장하는 별도 데이터 프레임을 만들고 비교
* 옵션 : 다음 사항은 선택사항입니다. 시간이 허용하는 범위 내에서 수행하세요.
    * 상위 N개 변수를 선정하여 모델링 및 성능 비교
        * 모델링에 항상 모든 변수가 필요한 것은 아닙니다.
        * 변수 중요도 상위 N개를 선정하여 모델링하고 타 모델과 성능을 비교하세요.
        * 상위 N개를 선택하는 방법은, 변수를 하나씩 늘려가며 모델링 및 성능 검증을 수행하여 적절한 지점을 찾는 것입니다.

## 1.환경설정

### (1) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler

In [14]:
# 라이브러리 불러오기
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

* 함수 생성

In [2]:
# 변수의 특성 중요도 계산하기
def plot_feature_importance(importance, names, result_only = False, topn = 'all'):
    feature_importance = np.array(importance)
    feature_name = np.array(names)

    data={'feature_name':feature_name,'feature_importance':feature_importance}
    fi_temp = pd.DataFrame(data)

    #변수의 특성 중요도 순으로 정렬하기
    fi_temp.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    fi_temp.reset_index(drop=True, inplace = True)

    if topn == 'all' :
        fi_df = fi_temp.copy()
    else :
        fi_df = fi_temp.iloc[:topn]

    #변수의 특성 중요도 그래프로 그리기
    if result_only == False :
        plt.figure(figsize=(10,20))
        sns.barplot(x='feature_importance', y='feature_name', data = fi_df)

        plt.xlabel('importance')
        plt.ylabel('feature name')
        plt.grid()

    return fi_df

### (2) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용
* 세부 요구사항
    - 전체 데이터 'data01_train.csv' 를 불러와 'data' 이름으로 저장합니다.
        - data에서 변수 subject는 삭제합니다.
    - 데이터프레임에 대한 기본 정보를 확인합니다.( .head(), .shape 등)

#### 1) 데이터 로딩

In [3]:
import joblib
data_path = r'C:\Users\user\kt에이블 학습\5주차\2024.03.27_미니프로젝트 3차_실습자료\data01_train.csv'
fi_path =  r'C:\Users\user\kt에이블 학습\5주차\2024.03.27_미니프로젝트 3차_실습자료\feature_importances.pkl'
data = pd.read_csv(data_path)
fi = joblib.load(fi_path)

In [4]:
data.drop('subject', axis=1, inplace=True)

In [5]:
data

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,0.288508,-0.009196,-0.103362,-0.988986,-0.962797,-0.967422,-0.989000,-0.962596,-0.965650,-0.929747,...,-0.487737,-0.816696,-0.042494,-0.044218,0.307873,0.072790,-0.601120,0.331298,0.165163,STANDING
1,0.265757,-0.016576,-0.098163,-0.989551,-0.994636,-0.987435,-0.990189,-0.993870,-0.987558,-0.937337,...,-0.237820,-0.693515,-0.062899,0.388459,-0.765014,0.771524,0.345205,-0.769186,-0.147944,LAYING
2,0.278709,-0.014511,-0.108717,-0.997720,-0.981088,-0.994008,-0.997934,-0.982187,-0.995017,-0.942584,...,-0.535287,-0.829311,0.000265,-0.525022,-0.891875,0.021528,-0.833564,0.202434,-0.032755,STANDING
3,0.289795,-0.035536,-0.150354,-0.231727,-0.006412,-0.338117,-0.273557,0.014245,-0.347916,0.008288,...,-0.004012,-0.408956,-0.255125,0.612804,0.747381,-0.072944,-0.695819,0.287154,0.111388,WALKING
4,0.394807,0.034098,0.091229,0.088489,-0.106636,-0.388502,-0.010469,-0.109680,-0.346372,0.584131,...,-0.157832,-0.563437,-0.044344,-0.845268,-0.974650,-0.887846,-0.705029,0.264952,0.137758,WALKING_DOWNSTAIRS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5876,0.277194,-0.012389,-0.131974,-0.994046,-0.940578,-0.917337,-0.994261,-0.932830,-0.908088,-0.936219,...,-0.305772,-0.690363,-0.034888,-0.261437,-0.391477,-0.877612,-0.912365,0.114009,0.080146,SITTING
5877,0.191568,0.013328,-0.105174,-0.126969,-0.121729,-0.327480,-0.192523,-0.109923,-0.295286,0.078644,...,-0.622617,-0.879215,0.721718,0.623151,0.866858,-0.445660,-0.690278,0.303194,-0.044188,WALKING_UPSTAIRS
5878,0.267981,-0.018348,-0.107440,-0.991303,-0.989881,-0.990313,-0.992386,-0.988852,-0.991237,-0.936099,...,-0.597610,-0.886851,0.060173,0.228739,0.684400,-0.216665,0.620363,-0.437247,-0.571840,LAYING
5879,0.212787,-0.048130,-0.121001,-0.041373,0.052449,-0.585361,-0.100714,0.023353,-0.554707,0.219814,...,0.196921,-0.053556,0.260880,0.551742,-0.943773,-0.862899,-0.718009,0.292856,0.024920,WALKING_UPSTAIRS


#### 2) 기본 정보 조회

In [ ]:
data.head()

In [6]:
data.shape

(5881, 562)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5881 entries, 0 to 5880
Columns: 562 entries, tBodyAcc-mean()-X to Activity
dtypes: float64(561), object(1)
memory usage: 25.2+ MB


In [8]:
data.describe()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
count,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,...,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000
mean,0.274811,-0.017799,-0.109396,-0.603138,-0.509815,-0.604058,-0.628151,-0.525944,-0.605374,-0.465490,...,0.126955,-0.305883,-0.623548,0.008524,-0.001185,0.009340,-0.007099,-0.491501,0.059299,-0.054594
std,0.067614,0.039422,0.058373,0.448807,0.501815,0.417319,0.424345,0.485115,0.413043,0.544995,...,0.249176,0.322808,0.310371,0.339730,0.447197,0.608190,0.476738,0.509069,0.297340,0.278479
min,-0.503823,-0.684893,-1.000000,-1.000000,-0.999844,-0.999667,-1.000000,-0.999419,-1.000000,-1.000000,...,-0.965725,-0.979261,-0.999765,-0.976580,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.980143
25%,0.262919,-0.024877,-0.121051,-0.992774,-0.977680,-0.980127,-0.993602,-0.977865,-0.980112,-0.936067,...,-0.021610,-0.541969,-0.845985,-0.122361,-0.294369,-0.481718,-0.373345,-0.811397,-0.018203,-0.141555
50%,0.277154,-0.017221,-0.108781,-0.943933,-0.844575,-0.856352,-0.948501,-0.849266,-0.849896,-0.878729,...,0.133887,-0.342923,-0.712677,0.010278,0.005146,0.011448,-0.000847,-0.709441,0.182893,0.003951
75%,0.288526,-0.010920,-0.098163,-0.242130,-0.034499,-0.262690,-0.291138,-0.068857,-0.268539,-0.013690,...,0.288944,-0.127371,-0.501158,0.154985,0.285030,0.499857,0.356236,-0.511330,0.248435,0.111932
max,1.000000,1.000000,1.000000,1.000000,0.916238,1.000000,1.000000,0.967664,1.000000,1.000000,...,0.946700,0.989538,0.956845,1.000000,1.000000,0.998702,0.996078,0.977344,0.478157,1.000000


## **2. 데이터 전처리**

* 가변수화, 데이터 분할, NaN 확인 및 조치, 스케일링 등 필요한 전처리를 수행한다. 


### (1) 데이터 분할1 : x, y

* 세부 요구사항
    - x, y로 분할합니다.

In [10]:
x = data.drop('Activity', axis=1)
y = data.loc[:, 'Activity']

### (2) 스케일링(필요시)


* 세부 요구사항
    - 스케일링을 필요로 하는 알고리즘 사용을 위해서 코드 수행
    - min-max 방식 혹은 standard 방식 중 한가지 사용.

In [12]:
mm_scaler = MinMaxScaler()
x_train_scaled = mm_scaler.fit_transform(x_train)
x_val_scaled = mm_scaler.transform(x_val)

### (3) 데이터분할2 : train, validation

* 세부 요구사항
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

In [11]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=40)

## **3. 기본 모델링**



* 세부 요구사항
    - 최소 4개 이상의 알고리즘을 적용하여 모델링을 수행한다. 
    - 각 알고리즘별로 전체 변수로 모델링, 상위 N개 변수를 선택하여 모델링을 수행하고 성능 비교를 한다.
    - (옵션) 알고리즘 중 1~2개에 대해서, 변수 중요도 상위 N개를 선정하여 모델링하고 타 모델과 성능을 비교.
        * 상위 N개를 선택하는 방법은, 변수를 하나씩 늘려가며 모델링 및 성능 검증을 수행하여 적절한 지점을 찾는 것이다.

### (1) 알고리즘1 : 

In [18]:
# KNN

rf = KNeighborsClassifier(n_neighbors=5)
rf.fit(x_train,y_train)

rf.score(x_val,y_val)

0.967714528462192

In [65]:
# 예측하기
y_pred_train = rf.predict(x_train)

In [66]:
# 훈련 데이터에 대한 예측 정확도를 계산합니다.
train_accuracy = accuracy_score(y_train, y_pred_train)
print("Training Accuracy:", train_accuracy)

# 훈련 데이터에 대한 분류 보고서를 출력하여 모델의 성능을 자세히 확인합니다.
print("Classification Report (Training Data):")
print(classification_report(y_train, y_pred_train))

Training Accuracy: 0.9749149659863946
Classification Report (Training Data):
                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       886
           SITTING       0.95      0.92      0.93       817
          STANDING       0.92      0.95      0.94       864
           WALKING       0.99      1.00      1.00       795
WALKING_DOWNSTAIRS       1.00      1.00      1.00       641
  WALKING_UPSTAIRS       1.00      1.00      1.00       701

          accuracy                           0.97      4704
         macro avg       0.98      0.98      0.98      4704
      weighted avg       0.98      0.97      0.97      4704



### (2) 알고리즘2 : 

In [29]:
## DecisionTree

# 선언하기
model = DecisionTreeClassifier(max_depth=50, random_state=1)
# 학습하기
model.fit(x_train, y_train)
model.score(x_val, y_val)

0.9286321155480034

In [52]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Decision Tree 모델을 생성하고 훈련합니다.
model = DecisionTreeClassifier(max_depth=50, random_state=1)
model.fit(x_train, y_train)

# 검증 데이터를 사용하여 모델의 정확도를 계산합니다.
validation_accuracy = model.score(x_val, y_val)
print("Validation Accuracy:", validation_accuracy)

# 훈련 데이터를 사용하여 예측을 수행합니다.
y_pred_train = model.predict(x_train)

# 훈련 데이터에 대한 예측 정확도를 계산합니다.
train_accuracy = accuracy_score(y_train, y_pred_train)
print("Training Accuracy:", train_accuracy)

# 훈련 데이터에 대한 분류 보고서를 출력하여 모델의 성능을 자세히 확인합니다.
print("Classification Report (Training Data):")
print(classification_report(y_train, y_pred_train))

Validation Accuracy: 0.9286321155480034
Training Accuracy: 1.0
Classification Report (Training Data):
                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       886
           SITTING       1.00      1.00      1.00       817
          STANDING       1.00      1.00      1.00       864
           WALKING       1.00      1.00      1.00       795
WALKING_DOWNSTAIRS       1.00      1.00      1.00       641
  WALKING_UPSTAIRS       1.00      1.00      1.00       701

          accuracy                           1.00      4704
         macro avg       1.00      1.00      1.00      4704
      weighted avg       1.00      1.00      1.00      4704



### (3) 알고리즘3 : 

In [28]:
# LogisticRegression 모델 선언 및 최대 반복 횟수 증가

model = LogisticRegression(max_iter=1000)

# 학습하기
model.fit(x_train, y_train)
model.score(x_val,y_val)

0.9813084112149533

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Logistic Regression 모델을 선언하고 최대 반복 횟수를 증가시킵니다.
model = LogisticRegression(max_iter=1000)

# 주어진 데이터를 사용하여 모델을 학습합니다.
model.fit(x_train, y_train)

# 검증 데이터를 사용하여 모델의 정확도를 계산합니다.
validation_accuracy = model.score(x_val, y_val)
print("Validation Accuracy:", validation_accuracy)

# 훈련 데이터를 사용하여 예측을 수행합니다.
y_pred_train = model.predict(x_train)

# 훈련 데이터에 대한 예측 정확도를 계산합니다.
train_accuracy = accuracy_score(y_train, y_pred_train)
print("Training Accuracy:", train_accuracy)

# 훈련 데이터에 대한 분류 보고서를 출력하여 모델의 성능을 자세히 확인합니다.
print("Classification Report (Training Data):")
print(classification_report(y_train, y_pred_train))

Validation Accuracy: 0.9813084112149533
Training Accuracy: 0.9931972789115646
Classification Report (Training Data):
                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       886
           SITTING       0.98      0.98      0.98       817
          STANDING       0.98      0.98      0.98       864
           WALKING       1.00      1.00      1.00       795
WALKING_DOWNSTAIRS       1.00      1.00      1.00       641
  WALKING_UPSTAIRS       1.00      1.00      1.00       701

          accuracy                           0.99      4704
         macro avg       0.99      0.99      0.99      4704
      weighted avg       0.99      0.99      0.99      4704



### (4) 알고리즘4 : 

In [58]:
# RandomForestClassifer
# 선언하기
model = RandomForestClassifier(max_depth=5, random_state=1)

# 학습하기
model.fit(x_train, y_train)
model.score(x_train, y_train)

# 검증 데이터를 사용하여 모델의 정확도를 계산합니다.
validation_accuracy = model.score(x_val, y_val)
print("Validation Accuracy:", validation_accuracy)

# 훈련 데이터를 사용하여 예측을 수행합니다.
y_pred_train = model.predict(x_train)

# 훈련 데이터에 대한 예측 정확도를 계산합니다.
train_accuracy = accuracy_score(y_train, y_pred_train)
print("Training Accuracy:", train_accuracy)

# 훈련 데이터에 대한 분류 보고서를 출력하여 모델의 성능을 자세히 확인합니다.
print("Classification Report (Training Data):")
print(classification_report(y_train, y_pred_train))

Validation Accuracy: 0.9439252336448598
Training Accuracy: 0.9457908163265306
Classification Report (Training Data):
                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       886
           SITTING       0.93      0.91      0.92       817
          STANDING       0.92      0.93      0.93       864
           WALKING       0.93      0.97      0.95       795
WALKING_DOWNSTAIRS       0.96      0.88      0.92       641
  WALKING_UPSTAIRS       0.93      0.97      0.95       701

          accuracy                           0.95      4704
         macro avg       0.95      0.94      0.94      4704
      weighted avg       0.95      0.95      0.95      4704



### (5) 알고리즘5 :

In [61]:
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report


# 클래스 레이블을 숫자로 변환
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# XGBoost 모델 선언
model = XGBClassifier(random_state=40)

# 학습
model.fit(x_train, y_train_encoded)

# 검증 데이터에 대한 정확도 평가
model.score(x_val, y_val_encoded)

0.994052676295667

### (6) 알고리즘6 :

In [62]:
model = LGBMClassifier(random_state=40)
model.fit(x_train.values, y_train.values)

model.score(x_val, y_val)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.032637 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 140146
[LightGBM] [Info] Number of data points in the train set: 4704, number of used features: 561
[LightGBM] [Info] Start training from score -1.669452
[LightGBM] [Info] Start training from score -1.750529
[LightGBM] [Info] Start training from score -1.694596
[LightGBM] [Info] Start training from score -1.777826
[LightGBM] [Info] Start training from score -1.993139
[LightGBM] [Info] Start training from score -1.903661
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

0.9915038232795242